## Import libraries

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import re
from datetime import date, datetime
import json
from collections import ChainMap

In [2]:
pd.set_option('display.max_rows', None, 'display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [3]:
!pwd

/home/xavi/Documents/CreditRisk/notebooks


## Import data

In [4]:
CURRENT_DATE = date(2017, 12, 1)

In [5]:
folder: Path = Path('/home/xavi/Documents/CreditRisk/data/')

In [6]:
df: pd.DataFrame = pd.read_csv(folder / 'raw/loan_data_2007_2014.csv', nrows=10_000)

In [7]:
df.head()

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,0,1077501,1296599,5000,5000,4975.0,36 months,10.65,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-11,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail.action?loan_id=1077501,Borrower added on 12/22/11 > I need to upgrade my business technologies.<br>,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-85,1.0,NaN,NaN,3.0,0.0,13648,83.7,9.0,f,0.0,0.0,5861.071414,5831.78,5000.00,861.07,0.00,0.00,0.00,Jan-15,171.62,NaN,Jan-16,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1077430,1314167,2500,2500,2500.0,60 months,15.27,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-11,Charged Off,n,https://www.lendingclub.com/browse/loanDetail.action?loan_id=1077430,Borrower added on 12/22/11 > I plan to use this money to finance the motorcycle i am looking at. I plan to have it paid off as soon as possible/when i sell my old bike. I only need this money because the deal im looking at is to good to pass up.<br><br> Borrower added on 12/22/11 > I plan to use this money to finance the motorcycle i am looking at. I plan to have it paid off as soon as possible/when i sell my old bike.I only need this money because the deal im looking at is to good to pass up. I have finished college with an associates degree in business and its takingmeplaces<br>,car,bike,309xx,GA,1.00,0.0,Apr-99,5.0,NaN,NaN,3.0,0.0,1687,9.4,4.0,f,0.0,0.0,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-13,119.66,NaN,Sep-13,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1077175,1313524,2400,2400,2400.0,36 months,15.96,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-11,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail.action?loan_id=1077175,NaN,small_business,real estate business,606xx,IL,8.72,0.0,Nov-01,2.0,NaN,NaN,2.0,0.0,2956,98.5,10.0,f,0.0,0.0,3003.653644,3003.65,2400.00,603.65,0.00,0.00,0.00,Jun-14,649.91,NaN,Jan-16,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1076863,1277178,10000,10000,10000.0,36 months,13.49,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-11,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail.action?loan_id=1076863,"Borrower added on 12/21/11 > to pay for property tax (borrow from friend, need to pay back) & central A/C need to be replace. I'm very sorry to let my loan expired last time.<br>",other,personel,917xx,CA,20.00,0.0,Feb-96,1.0,35.0,NaN,10.0,0.0,5598,21.0,37.0,f,0.0,0.0,12226.302210,12226.30,10000.00,2209.33,16.97,0.00,0.00,Jan-15,357.48,NaN,Jan-15,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1075358,1311748,3000,3000,3000.0,60 months,12.69,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-11,Current,n,https://www.lendingclub.com/browse/loanDetail.action?loan_id=1075358,"Borrower added on 12/21/11 > I plan on combining three large interest bills together and f

In [8]:
len(df)

10000

In [9]:
len(df['id'].unique())

10000

In [10]:
df = df.set_index('id')

In [279]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 1077501 to 840015
Data columns (total 77 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   member_id                      10000 non-null  int64         
 1   loan_amnt                      10000 non-null  float64       
 2   funded_amnt                    10000 non-null  float64       
 3   funded_amnt_inv                10000 non-null  float64       
 4   term                           10000 non-null  category      
 5   int_rate                       10000 non-null  float64       
 6   installment                    10000 non-null  float64       
 7   grade                          10000 non-null  category      
 8   sub_grade                      10000 non-null  category      
 9   emp_title                      9348 non-null   string        
 10  home_ownership                 10000 non-null  category      
 11  annual_inc   

In [12]:
schema: pd.DataFrame = pd.read_json(folder / 'curated/data_dictionary.json', orient='records')

In [13]:
schema.head()

,LoanStatNew,Description
0,addr_state,The state provided by the borrower in the loan application
1,annual_inc,The self-reported annual income provided by the borrower during registration.
2,annual_inc_joint,The combined self-reported annual income provided by the co-borrowers during registration
3,application_type,Indicates whether the loan is an individual application or a joint application with two co-borrowers
4,collection_recovery_fee,post charge off collection fee


## Check data types

In [14]:
df.dtypes

Unnamed: 0                       int64
member_id                        int64
loan_amnt                        int64
funded_amnt                      int64
funded_amnt_inv                float64
term                            object
int_rate                       float64
installment                    float64
grade                           object
sub_grade                       object
emp_title                       object
emp_length                      object
home_ownership                  object
annual_inc                     float64
verification_status             object
issue_d                         object
loan_status                     object
pymnt_plan                      object
url                             object
desc                            object
purpose                         object
title                           object
zip_code                        object
addr_state                      object
dti                            float64
delinq_2yrs              

In [15]:
data_dict: pd.DataFrame = pd.merge(schema, df.dtypes.rename('current_type'), left_on='LoanStatNew', right_index=True, how='outer', indicator=True)

In [16]:
data_dict['target_type'] = pd.Series([], dtype='object')

In [17]:
data_dict['changed'] = pd.Series([False] * len(data_dict), dtype='bool')

In [18]:
data_dict['_merge'] = data_dict['_merge'].replace({'both': 'both', 'left_only': 'schema', 'right_only': 'data'})

In [19]:
data_dict.sort_values(by='current_type')

,LoanStatNew,Description,current_type,_merge,target_type,changed
31.0,loan_amnt,"The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value.",int64,both,NaN,False
NaN,Unnamed: 0,NaN,int64,data,NaN,NaN
46.0,revol_bal,Total credit revolving balance,int64,both,NaN,False
15.0,funded_amnt,The total amount committed to that loan at that point in time.,int64,both,NaN,False
33.0,member_id,A unique LC assigned Id for the borrower member.,int64,both,NaN,False
41.0,policy_code,publicly available policy_code=1\nnew products not publicly available policy_code=2,int64,both,NaN,False
60.0,open_acc_6m,Number of open trades in last 6 months,float64,both,NaN,False
56.0,total_rec_prncp,Principal received to date,float64,both,NaN,False
55.0,total_rec_late_fee,Late fees received to date,float64,both,NaN,False
54.0,total_rec_int,Interest received to date,float64,both,NaN,False


Drop Column:

In [20]:
df = df.drop('Unnamed: 0', axis=1)

In [21]:
df['verification_status'].value_counts()

verification_status
Verified           3881
Source Verified    3069
Not Verified       3050
Name: count, dtype: int64

In [22]:
data_dict.loc[data_dict['LoanStatNew'] == 'verification_status', 'target_type'] = 'categorical'

In [23]:
data_dict.loc[data_dict['LoanStatNew'] == 'verification_status', 'changed'] = False

In [24]:
df['verification_status_joint'].value_counts()

Series([], Name: count, dtype: int64)

In [25]:
df['total_rev_hi_lim'].value_counts()

Series([], Name: count, dtype: int64)

In [26]:
df['purpose'].value_counts()

purpose
debt_consolidation    5155
credit_card           1503
other                  865
home_improvement       630
small_business         439
major_purchase         365
car                    298
wedding                217
medical                176
moving                 126
house                  113
vacation                91
renewable_energy        22
Name: count, dtype: int64

In [27]:
data_dict.loc[data_dict['LoanStatNew'] == 'purpose', 'target_type'] = 'categorical'

In [28]:
df['next_pymnt_d'].head()

id
1077501       NaN
1077430       NaN
1077175       NaN
1076863       NaN
1075358    Feb-16
Name: next_pymnt_d, dtype: object

In [29]:
data_dict.loc[data_dict['LoanStatNew'] == 'next_pymnt_d', 'target_type'] = 'date'

In [30]:
df['last_credit_pull_d'].head()

id
1077501    Jan-16
1077430    Sep-13
1077175    Jan-16
1076863    Jan-15
1075358    Jan-16
Name: last_credit_pull_d, dtype: object

In [31]:
data_dict.loc[data_dict['LoanStatNew'] == 'last_credit_pull_d', 'target_type'] = 'date'

In [32]:
data_dict['sample'] = data_dict['LoanStatNew']\
                        .apply(lambda v: df[v].head(30).to_list() if v in df.columns.values else np.nan)

In [33]:
data_dict.sort_values(by='_merge')

,LoanStatNew,Description,current_type,_merge,target_type,changed,sample
27.0,last_fico_range_high,The upper boundary range the borrower’s last FICO pulled belongs to.,NaN,schema,NaN,False,NaN
28.0,last_fico_range_low,The lower boundary range the borrower’s last FICO pulled belongs to.,NaN,schema,NaN,False,NaN
58.0,verified_status_joint,"Indicates if the co-borrowers' joint income was verified by LC, not verified, or if the income source was verified",NaN,schema,NaN,False,NaN
79.0,None,* Employer Title replaces Employer Name for all loans listed after 9/23/2013,NaN,schema,NaN,False,NaN
78.0,None,None,NaN,schema,NaN,False,NaN
19.0,id,A unique LC assigned ID for the loan listing.,NaN,schema,NaN,False,NaN
14.0,fico_range_low,The lower boundary range the borrower’s FICO at loan origination belongs to.,NaN,schema,NaN,False,NaN
13.0,fico_range_high,The upper boundary range the borrower’s FICO at loan origination belongs to.,NaN,schema,NaN,False,NaN
24.0,is_inc_v,"Indicates if income was verified by LC, not verified, or if the income source was verified",NaN,schema,NaN,False,NaN
71.0,total_rev_hi_lim,Total revolving high credit/credit limit,NaN,schema,NaN,False,NaN


In [34]:
for col in filter(lambda c: re.search('_d$', c) , df.columns.values):
    data_dict.loc[data_dict['LoanStatNew'] == col, 'target_type'] = 'date'

In [35]:
data_dict.sort_values(by='_merge')

,LoanStatNew,Description,current_type,_merge,target_type,changed,sample
27.0,last_fico_range_high,The upper boundary range the borrower’s last FICO pulled belongs to.,NaN,schema,NaN,False,NaN
28.0,last_fico_range_low,The lower boundary range the borrower’s last FICO pulled belongs to.,NaN,schema,NaN,False,NaN
58.0,verified_status_joint,"Indicates if the co-borrowers' joint income was verified by LC, not verified, or if the income source was verified",NaN,schema,NaN,False,NaN
79.0,None,* Employer Title replaces Employer Name for all loans listed after 9/23/2013,NaN,schema,NaN,False,NaN
78.0,None,None,NaN,schema,NaN,False,NaN
19.0,id,A unique LC assigned ID for the loan listing.,NaN,schema,NaN,False,NaN
14.0,fico_range_low,The lower boundary range the borrower’s FICO at loan origination belongs to.,NaN,schema,NaN,False,NaN
13.0,fico_range_high,The upper boundary range the borrower’s FICO at loan origination belongs to.,NaN,schema,NaN,False,NaN
24.0,is_inc_v,"Indicates if income was verified by LC, not verified, or if the income source was verified",NaN,schema,NaN,False,NaN
71.0,total_rev_hi_lim,Total revolving high credit/credit limit,NaN,schema,NaN,False,NaN


In [36]:
data_dict.loc[data_dict['LoanStatNew'] == 'term', 'target_type'] ='categorical'

In [37]:
data_dict.loc[data_dict['LoanStatNew'] == 'sub_grade', 'target_type'] = 'categorical'

In [124]:
data_dict.loc[data_dict['LoanStatNew'] == 'total_rec_late_fee', 'target_type'] = 'float64'

In [39]:
data_dict.loc[data_dict['LoanStatNew'] == 'total_acc', 'target_type'] = 'int64'

In [126]:
data_dict.loc[data_dict['LoanStatNew'] == 'open_il_24m', 'target_type'] = 'int16'

In [125]:
data_dict.loc[data_dict['LoanStatNew'] == 'open_il_6m', 'target_type'] = 'int16'

In [123]:
data_dict.loc[data_dict['LoanStatNew'] == 'acc_now_delinq', 'target_type'] = 'int16'

Create an int version for emp_length:

In [43]:
df['emp_length_int'] = df['emp_length']\
    .str.replace('< 1 year', str(0))\
    .str.extract(r'(?P<years>[0-9]{1,2})')\
    .fillna(0)\
    .astype(np.int8)

In [44]:
data_dict.loc[data_dict['LoanStatNew'] == 'emp_length', 'target_type'] = 'categorical'
data_dict.loc[data_dict['LoanStatNew'] == 'emp_length', 'changed'] = True

earliest_cr_line:
- Convert into date
- Calculate time gone by until current date in months
- Fix negatives values

In [45]:
df['months_since_earliest_cr_line'] = np.round(
( pd.to_datetime(CURRENT_DATE) -
    df.assign(year=df['earliest_cr_line'].str.extract(r'(?P<year>[0-9]{2})$').astype(np.int8).map(lambda y: 2000 + y if 0 <= y <= 17 else 1900 + y),
              month=df['earliest_cr_line'].str.extract(r'(?P<month>[A-Z]{1}[a-z]{2})'))\
       .apply(lambda d: datetime.strptime(d['month'] + '-' + str(d['year']), '%b-%Y'), axis=1)
  ) / np.timedelta64(30, 'D')
).astype('int64')


In [95]:
data_dict.loc[data_dict['LoanStatNew'] == 'earliest_cr_line', 'target_type'] = 'date'
data_dict.loc[data_dict['LoanStatNew'] == 'earliest_cr_line', 'changed'] = True

Create an int version for term

In [47]:
df['term'].unique()

array([' 36 months', ' 60 months'], dtype=object)

In [48]:
df['term_int'] = df['term'].str.extract(r'(?P<year>[0-9]{1,2})').astype('int8')

In [73]:
data_dict.loc[data_dict['LoanStatNew'] == 'term', 'target_type'] = 'categorical'
data_dict.loc[data_dict['LoanStatNew'] == 'term', 'changed'] = True

In [74]:
df['term'] = df['term'].astype('category')

In [50]:
df = df.drop('emp_length', axis=1)

Calculate months since issuance:

In [97]:
df['months_since_issue_d'] = np.round(
( pd.to_datetime(CURRENT_DATE) -
    df.assign(year=df['issue_d'].str.extract(r'(?P<year>[0-9]{2})$').astype(np.int8).map(lambda y: 2000 + y if 0 <= y <= 17 else 1900 + y),
              month=df['issue_d'].str.extract(r'(?P<month>[A-Z]{1}[a-z]{2})'))\
       .apply(lambda d: datetime.strptime(d['month'] + '-' + str(d['year']), '%b-%Y'), axis=1)
  ) / np.timedelta64(30, 'D')
).astype('int16')

In [98]:
df['months_since_issue_d'].describe()

count    10000.00000
mean        74.79330
std          1.34928
min         73.00000
25%         74.00000
50%         75.00000
75%         76.00000
max         77.00000
Name: months_since_issue_d, dtype: float64

In [53]:
df['issue_date'] = df.assign(year=df['issue_d'].str.extract(r'(?P<year>[0-9]{2})$').astype(np.int8).map(lambda y: 2000 + y if 0 <= y <= 17 else 1900 + y),
                            month=df['issue_d'].str.extract(r'(?P<month>[A-Z]{1}[a-z]{2})'))\
                      .apply(lambda d: datetime.strptime(d['month'] + '-' + str(d['year']), '%b-%Y'), axis=1)

Convert into category:

In [54]:
df['verification_status'] = df['verification_status'].astype('category')
df['term'] = df['term'].astype('category')
df['sub_grade'] = df['sub_grade'].astype('category')
df['purpose'] = df['purpose'].astype('category')
df['grade'] = df['grade'].astype('category')
df['addr_state'] = df['addr_state'].astype('category')
df['application_type'] = df['application_type'].astype('category')
df['loan_status'] = df['loan_status'].astype('category')
df['home_ownership'] = df['home_ownership'].astype('category')
df['pymnt_plan'] = df['pymnt_plan'].astype('category')
df['initial_list_status'] = df['initial_list_status'].astype('category')
df['initial_list_status'] = df['policy_code'].astype('category')

In [55]:
data_dict.loc[data_dict['LoanStatNew'] == 'grade', 'target_type'] = 'categorical'
data_dict.loc[data_dict['LoanStatNew'] == 'addr_state', 'target_type'] = 'categorical'
data_dict.loc[data_dict['LoanStatNew'] == 'application_type', 'target_type'] = 'categorical'
data_dict.loc[data_dict['LoanStatNew'] == 'loan_status', 'target_type'] = 'categorical'
data_dict.loc[data_dict['LoanStatNew'] == 'home_ownership', 'target_type'] = 'categorical'
data_dict.loc[data_dict['LoanStatNew'] == 'pymnt_plan', 'target_type'] = 'categorical'
data_dict.loc[data_dict['LoanStatNew'] == 'initial_list_status', 'target_type'] = 'categorical'
data_dict.loc[data_dict['LoanStatNew'] == 'policy_code', 'target_type'] = 'categorical'

In [56]:
df['verification_status_joint'].unique()

array([nan])

To int

In [57]:
df['zip_code'] = df['zip_code'].str.extract(r'(?P<zip>[0-9]{3})').astype('int16')
data_dict.loc[data_dict['LoanStatNew'] == 'zip_code', 'target_type'] = 'int16'

In [58]:
df['total_acc'] = df['total_acc'].astype('int8')
data_dict.loc[data_dict['LoanStatNew'] == 'total_acc', 'target_type'] = 'int8'

In [59]:
df['total_acc'].describe()

count    10000.000000
mean        22.540600
std         11.098264
min          3.000000
25%         14.000000
50%         21.000000
75%         29.000000
max         79.000000
Name: total_acc, dtype: float64

In [60]:
df['inq_last_6mths'] = df['inq_last_6mths'].astype('int8')
data_dict.loc[data_dict['LoanStatNew'] == 'inq_last_6mths', 'target_type'] = 'int8'

In [61]:
df.memory_usage(deep=True)

Index                              80000
member_id                          80000
loan_amnt                          80000
funded_amnt                        80000
funded_amnt_inv                    80000
term                               10226
int_rate                           80000
installment                        80000
grade                              10650
sub_grade                          12857
emp_title                         651605
home_ownership                     10270
annual_inc                         80000
verification_status                10290
issue_d                           550000
loan_status                        10728
pymnt_plan                         10158
url                              1164373
desc                             2388956
purpose                            11323
title                             653983
zip_code                           20000
addr_state                         13367
dti                                80000
delinq_2yrs     

In [62]:
pd.get_dummies(df['grade'], prefix='grade').head()

,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G
id,,,,,,,
1077501,False,True,False,False,False,False,False
1077430,False,False,True,False,False,False,False
1077175,False,False,True,False,False,False,False
1076863,False,False,True,False,False,False,False
1075358,False,True,False,False,False,False,False


In [71]:
description_mask: pd.Series = (data_dict['Description'].str.contains('[N,n]umber')) & (~data_dict['Description'].isnull())

In [142]:
data_dict[description_mask]

,LoanStatNew,Description,current_type,_merge,target_type,changed,sample
5.0,collections_12_mths_ex_med,Number of collections in 12 months excluding medical collections,float64,both,int8,False,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
6.0,delinq_2yrs,The number of 30+ days past-due incidences of delinquency in the borrower's credit file for the past 2 years,float64,both,int8,False,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 1.0, 0.0]"
21.0,inq_last_6mths,The number of inquiries in past 6 months (excluding auto and mortgage inquiries),float64,both,int8,False,"[1.0, 5.0, 2.0, 1.0, 0.0, 3.0, 1.0, 2.0, 2.0, 0.0, 2.0, 0.0, 1.0, 2.0, 2.0, 1.0, 2.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0]"
34.0,mths_since_last_delinq,The number of months since the borrower's last delinquency.,float64,both,int8,False,"[nan, nan, nan, 35.0, 38.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 61.0, nan, 8.0, nan, nan, nan, nan, nan, nan, nan, nan, 20.0, 18.0, nan]"
36.0,mths_since_last_record,The number of months since the last public record.,float64,both,NaN,False,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]"
38.0,open_acc,The number of open credit lines in the borrower's credit file.,float64,both,NaN,False,"[3.0, 3.0, 2.0, 10.0, 15.0, 9.0, 7.0, 4.0, 11.0, 2.0, 14.0, 12.0, 4.0, 11.0, 14.0, 11.0, 14.0, 20.0, 4.0, 8.0, 8.0, 7.0, 8.0, 6.0, 7.0, 8.0, 7.0, 14.0, 12.0, 8.0]"
42.0,pub_rec,Number of derogatory public records,float64,both,NaN,False,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
49.0,term,The number of payments on the loan. Values are in months and can be either 36 or 60.,object,both,categorical,True,"[ 36 months, 60 months, 36 months, 36 months, 60 months, 36 months, 60 months, 36 months, 60 months, 60 months, 60 months, 36 months, 36 months, 36 months, 36 months, 36 months, 36 months, 36 months, 36 months, 36 months, 60 months, 36 months, 36 months, 36 months, 36 months, 36 months, 36 months, 60 months, 36 months, 36 months]"
51.0,total_acc,The total number of credit lines currently in the borrower's credit file,float64,both,int8,False,"[9.0, 4.0, 10.0, 37.0, 38.0, 12.0, 11.0, 4.0, 13.0, 3.0, 23.0, 34.0, 9.0, 11.0, 29.0, 23.0, 28.0, 42.0, 14.0, 28.0, 22.0, 38.0, 21.0, 17.0, 7.0, 31.0, 11.0, 22.0, 44.0, 12.0]"
59.0,zip_code,The first 3 numbers of the zip code provided by the borrower in the loan application.,object,both,int16,False,"[860xx, 309xx, 606xx, 917xx, 972xx, 852xx, 280xx, 900xx, 958xx, 774xx, 853xx, 913xx, 245xx, 606xx, 951xx, 641xx, 921xx, 067xx, 890xx, 921xx, 770xx, 335xx, 799xx, 917xx, 900xx, 605xx, 103xx, 150xx, 326xx, 564xx]"


In [77]:
df['collections_12_mths_ex_med'] = df['collections_12_mths_ex_med'].astype('int8')
data_dict.loc[data_dict['LoanStatNew'] == 'collections_12_mths_ex_med', 'target_type'] = 'int8'

In [79]:
df['delinq_2yrs'] = df['delinq_2yrs'].astype('int8')
data_dict.loc[data_dict['LoanStatNew'] == 'delinq_2yrs', 'target_type'] = 'int8'

In [144]:
df['mths_since_last_delinq'] = df['mths_since_last_delinq'].fillna(0).astype('int8')
data_dict.loc[data_dict['LoanStatNew'] == 'mths_since_last_delinq', 'target_type'] = 'float64'

In [127]:
df.memory_usage(deep=True)

Index                              80000
member_id                          80000
loan_amnt                          80000
funded_amnt                        80000
funded_amnt_inv                    80000
term                               10226
int_rate                           80000
installment                        80000
grade                              10650
sub_grade                          12857
emp_title                         667253
home_ownership                     10270
annual_inc                         80000
verification_status                10290
issue_d                            80000
loan_status                        10728
pymnt_plan                         10158
url                              1164373
desc                             2388956
purpose                            11323
title                             653983
zip_code                           20000
addr_state                         13367
dti                                80000
delinq_2yrs     

In [89]:
df['title'].astype('string')._memory_usage(deep=True)/ df['title'].memory_usage(deep=True)

0.8910056499946184

In [90]:
df['title'] = df['title'].astype('string')
data_dict.loc[data_dict['LoanStatNew'] == 'title', 'target_type'] = 'string'

In [91]:
df['url'] = df['url'].astype('string')
data_dict.loc[data_dict['LoanStatNew'] == 'url', 'target_type'] = 'string'

In [130]:
df['emp_title'] = df['emp_title'].astype('string')
data_dict.loc[data_dict['LoanStatNew'] == 'emp_title', 'target_type'] = 'string'

In [129]:
df['desc'] = df['desc'].astype('string')
data_dict.loc[data_dict['LoanStatNew'] == 'desc', 'target_type'] = 'string'

In [228]:
def parse_date(d: str) -> datetime:
    # nan is of float type
    if d != '':
        year: str = int(d[-2:])
        full_year: int = 2000 + year if 0 <= year <= 17 else 1900 + year
    
        return datetime.strptime(d[:3] + '-' + str(full_year), '%b-%Y')
    
    return np.nan

In [114]:
df['next_pymnt_d'] = df['next_pymnt_d'].apply(parse_date)
df['last_pymnt_d'] = df['last_pymnt_d'].apply(parse_date)
df['last_credit_pull_d'] = df['last_credit_pull_d'].apply(parse_date)

In [119]:
df['issue_d'] = df['issue_d'].apply(parse_date)
df['earliest_cr_line'] = df['earliest_cr_line'].apply(parse_date)

In [120]:
df.memory_usage(deep=True)

Index                              80000
member_id                          80000
loan_amnt                          80000
funded_amnt                        80000
funded_amnt_inv                    80000
term                               10226
int_rate                           80000
installment                        80000
grade                              10650
sub_grade                          12857
emp_title                         667253
home_ownership                     10270
annual_inc                         80000
verification_status                10290
issue_d                            80000
loan_status                        10728
pymnt_plan                         10158
url                              1164373
desc                             2388956
purpose                            11323
title                             653983
zip_code                           20000
addr_state                         13367
dti                                80000
delinq_2yrs     

In [134]:
data_dict

LoanStatNew  \
0.0                    addr_state   
1.0                    annual_inc   
2.0              annual_inc_joint   
3.0              application_type   
4.0       collection_recovery_fee   
5.0    collections_12_mths_ex_med   
6.0                   delinq_2yrs   
7.0                          desc   
8.0                           dti   
9.0                     dti_joint   
10.0             earliest_cr_line   
11.0                   emp_length   
12.0                    emp_title   
13.0              fico_range_high   
14.0               fico_range_low   
15.0                  funded_amnt   
16.0              funded_amnt_inv   
17.0                        grade   
18.0               home_ownership   
19.0                           id   
20.0          initial_list_status   
21.0               inq_last_6mths   
22.0                  installment   
23.0                     int_rate   
24.0                     is_inc_v   
25.0                      issue_d   
26.0           last_credit_pull_d   
27.0         last_fico_range_high   
28.0          last_fico_range_low   
29.0              last_pymnt_amnt   
30.0                 last_pymnt_d   
31.0                    loan_amnt   
32.0                  loan_status   
33.0                    member_id   
34.0       mths_since_last_delinq   
35.0  mths_since_last_major_derog   
36.0       mths_since_last_record   
37.0                 next_pymnt_d   
38.0                     open_acc   
39.0                    out_prncp   
40.0                out_prncp_inv   
41.0                  policy_code   
42.0                      pub_rec   
43.0                      purpose   
44.0                   pymnt_plan   
45.0                   recoveries   
46.0                    revol_bal   
47.0                   revol_util   
48.0                    sub_grade   
49.0                         term   
50.0                        title   
51.0                    total_acc   
52.0                  total_pymnt   
53.0              total_pymnt_inv   
54.0                total_rec_int   
55.0           total_rec_late_fee   
56.0              total_rec_prncp   
57.0                          url   
58.0        verified_status_joint   
59.0                     zip_code   
60.0                  open_acc_6m   
61.0                   open_il_6m   
62.0                  open_il_12m   
63.0                  open_il_24m   
64.0           mths_since_rcnt_il   
65.0                 total_bal_il   
66.0                      il_util   
67.0                  open_rv_12m   
68.0                  open_rv_24m   
69.0                   max_bal_bc   
70.0                     all_util   
71.0           total_rev_hi_lim     
72.0                       inq_fi   
73.0                  total_cu_tl   
74.0                 inq_last_12m   
75.0               acc_now_delinq   
76.0                 tot_coll_amt   
77.0                  tot_cur_bal   
78.0                         None   
79.0                         None   
NaN                    Unnamed: 0   
NaN           verification_status   
NaN     verification_status_joint   
NaN              total_rev_hi_lim   

                                                                                                                                                                                                             Description  \
0.0                                                                                                                                                           The state provided by the borrower in the loan application   
1.0                                                                                                                                        The self-reported annual income provided by the borrower during registration.   
2.0                                                                                                                            The combined self-reported annual income provided by the co-borrowers during registration

In [132]:
df['loan_amnt'] = df['loan_amnt'].astype('float64')
data_dict.loc[data_dict['LoanStatNew'] == 'loan_amnt', 'target_type'] = 'float64'

df['revol_bal'] = df['revol_bal'].astype('float64')
data_dict.loc[data_dict['LoanStatNew'] == 'revol_bal', 'target_type'] = 'float64'

In [133]:
df['funded_amnt'] = df['funded_amnt'].astype('float64')
data_dict.loc[data_dict['LoanStatNew'] == 'funded_amnt', 'target_type'] = 'float64'

In [281]:
data_dict['LoanStatNew'].apply(lambda v: (df[v].isnull().sum() if v in df.columns.values else np.nan) < 10_000)

0.0      True
1.0      True
2.0     False
3.0      True
4.0      True
5.0      True
6.0      True
7.0      True
8.0      True
9.0     False
10.0     True
11.0    False
12.0     True
13.0    False
14.0    False
15.0     True
16.0     True
17.0     True
18.0     True
19.0    False
20.0     True
21.0     True
22.0     True
23.0     True
24.0    False
25.0     True
26.0     True
27.0    False
28.0    False
29.0     True
30.0     True
31.0     True
32.0     True
33.0     True
34.0     True
35.0    False
36.0     True
37.0     True
38.0     True
39.0     True
40.0     True
41.0     True
42.0     True
43.0     True
44.0     True
45.0     True
46.0     True
47.0     True
48.0     True
49.0     True
50.0     True
51.0     True
52.0     True
53.0     True
54.0     True
55.0     True
56.0     True
57.0     True
58.0    False
59.0     True
60.0    False
61.0    False
62.0    False
63.0    False
64.0    False
65.0    False
66.0    False
67.0    False
68.0    False
69.0    False
70.0    False
71.0  

In [284]:
df['open_acc_6m'].isnull().sum()

10000

In [286]:
data_dict.loc[data_dict['LoanStatNew'] == 'zip_code', 'target_type'] = 'string'

In [143]:
type_mask: pd.Series = (data_dict['current_type'] == 'float64') & (data_dict['target_type'].isnull()) \
                       & (data_dict['Description'].str.contains('[N,n]umber'))

data_dict[type_mask]

,LoanStatNew,Description,current_type,_merge,target_type,changed,sample
36.0,mths_since_last_record,The number of months since the last public record.,float64,both,NaN,False,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]"
38.0,open_acc,The number of open credit lines in the borrower's credit file.,float64,both,NaN,False,"[3.0, 3.0, 2.0, 10.0, 15.0, 9.0, 7.0, 4.0, 11.0, 2.0, 14.0, 12.0, 4.0, 11.0, 14.0, 11.0, 14.0, 20.0, 4.0, 8.0, 8.0, 7.0, 8.0, 6.0, 7.0, 8.0, 7.0, 14.0, 12.0, 8.0]"
42.0,pub_rec,Number of derogatory public records,float64,both,NaN,False,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
60.0,open_acc_6m,Number of open trades in last 6 months,float64,both,NaN,False,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]"
62.0,open_il_12m,Number of installment accounts opened in past 12 months,float64,both,NaN,False,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]"
67.0,open_rv_12m,Number of revolving trades opened in past 12 months,float64,both,NaN,False,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]"
68.0,open_rv_24m,Number of revolving trades opened in past 24 months,float64,both,NaN,False,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]"
72.0,inq_fi,Number of personal finance inquiries,float64,both,NaN,False,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]"
73.0,total_cu_tl,Number of finance trades,float64,both,NaN,False,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]"
74.0,inq_last_12m,Number of credit inquiries in past 12 months,float64,both,NaN,False,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]"


In [287]:
data_dict['target_type'] = data_dict['target_type'].apply(lambda v: 'category' if v == 'categorical' else v)

In [288]:
data_dict['target_type'] =data_dict['target_type'].mask(data_dict['target_type'].isna(), data_dict['current_type'])

In [292]:
existing_columns_mask: pd.Series = (data_dict['_merge'].isin(['both', 'data'])) & (~data_dict['target_type'].str.contains(r'[0-9]{1,2}'))

In [165]:
data_dict['target_type']= data_dict['target_type'].apply(str)

In [293]:
target_types: list = data_dict[existing_columns_mask][['LoanStatNew', 'target_type']].apply(lambda s: {s['LoanStatNew']: s['target_type']}, axis=1).to_list()

In [294]:
target_types

[{'addr_state': 'category'},
 {'application_type': 'category'},
 {'desc': 'string'},
 {'earliest_cr_line': 'date'},
 {'emp_length': 'category'},
 {'emp_title': 'string'},
 {'grade': 'category'},
 {'home_ownership': 'category'},
 {'initial_list_status': 'category'},
 {'issue_d': 'date'},
 {'last_credit_pull_d': 'date'},
 {'last_pymnt_d': 'date'},
 {'loan_status': 'category'},
 {'next_pymnt_d': 'date'},
 {'policy_code': 'category'},
 {'purpose': 'category'},
 {'pymnt_plan': 'category'},
 {'sub_grade': 'category'},
 {'term': 'category'},
 {'title': 'string'},
 {'url': 'string'},
 {'zip_code': 'string'},
 {'verification_status': 'category'}]

In [295]:
with open(folder / 'curated/data_types.json', 'w') as f:
    json.dump(target_types, f)

## Load file with new inferred types

In [6]:
with open(folder / 'curated/data_types.json', 'r') as f:
    types: list[dict] = json.load(f)

In [7]:
def parse_date(d: str) -> datetime:
    # nan is of float type
    if d != '':
        year: str = int(d[-2:])
        full_year: int = 2000 + year if 0 <= year <= 17 else 1900 + year
    
        return datetime.strptime(d[:3] + '-' + str(full_year), '%b-%Y')
    
    return np.nan

In [8]:
def zip_code(zc: str) -> np.int16:
    if zc != '':
        return np.int16(zc[:3])
    
    return np.nan

In [9]:
def transform_post_load(fr: pd.DataFrame) -> pd.DataFrame:
    fr['zip_code'] = fr['zip_code'].astype('category')
    fr = fr.drop('Unnamed: 0', axis=1)
    fr = fr.set_index('id')
    return fr

In [10]:
types_no_dates: list[dict] = list(filter(lambda v: 'date' not in list(v.values()),types))
types_dates: list[dict] = list(filter(lambda v: 'date' in list(v.values()),types))

In [11]:
types_no_dates = dict(ChainMap(*types_no_dates))

In [12]:
types_dates = dict(ChainMap(*types_dates))
for key in types_dates:
    types_dates[key] = parse_date

In [13]:
del types_no_dates['zip_code']

In [14]:
types_dates['zip_code'] = zip_code

In [15]:
new_df: pd.DataFrame = pd.read_csv(folder / 'raw/loan_data_2007_2014.csv',
    converters=types_dates, dtype=types_no_dates)

In [16]:
new_df = new_df.pipe(transform_post_load)

## Adding columns
- Create an int version for emp_length
- Create an int version for term
- months_since_issue_d
- earliest_cr_line: Calculate time gone by until current date in months

In [22]:
def add_columns(fr: pd.DataFrame) -> pd.DataFrame:
    fr['emp_length_int'] = fr['emp_length']\
        .str.replace('< 1 year', str(0))\
        .str.extract(r'(?P<years>[0-9]{1,2})')\
        .fillna(0)\
        .astype(np.int8)
    
    fr['term_int'] = fr['term'].str.extract(r'(?P<year>[0-9]{1,2})').astype('int8')
    
    fr['months_since_issue_d'] = np.round(
            ( pd.to_datetime(CURRENT_DATE) - fr['issue_d']) / np.timedelta64(30, 'D')
        ).astype('int16')
    
    fr['months_since_earliest_cr_line'] = np.round(
            ( pd.to_datetime(CURRENT_DATE) - fr['earliest_cr_line']) / np.timedelta64(30, 'D')
        ).astype('int16')
    
    return fr

In [24]:
new_df = new_df.pipe(add_columns)

## Filling nulls

- annual_inc: replace with mean of non-missing values.
- delinq_2yrs: replace with 0.
- inq_last_6mnths: replace with 0.
- open_acc: replace with 0.
- pub_rec: replace with 0.
- total_acc: replace with 0
- acc_now_delinq: replace with 0
- total_rev_hi_lim: fill with funded_amnt.
- emp_lenght_int: replace with 0.
- mnths_since_earliest_cr_line: replace with 0.

In [44]:
def fill_values(fr: pd.DataFrame) -> pd.DataFrame:
    fr['annual_inc'] = fr['annual_inc'].fillna(fr[~fr['annual_inc'].isnull()]['annual_inc'].mean())
    
    fr['delinq_2yrs'] = fr['delinq_2yrs'].fillna(0).astype('int16')
    fr['inq_last_6mths'] = fr['inq_last_6mths'].fillna(0).astype('int16')
    fr['open_acc'] = fr['open_acc'].fillna(0)
    fr['pub_rec'] = fr['pub_rec'].fillna(0)
    fr['total_acc'] = fr['total_acc'].fillna(0)
    fr['acc_now_delinq'] = fr['acc_now_delinq'].fillna(0)
    
    fr['total_rev_hi_lim'] = fr['total_rev_hi_lim'].mask(fr['total_rev_hi_lim'].isnull(), fr['funded_amnt'])
    
    fr['emp_length_int'] = fr['emp_length_int'].fillna(0)
    fr['months_since_earliest_cr_line'] = fr['months_since_earliest_cr_line'].fillna(0)
    return fr

In [45]:
new_df = new_df.pipe(fill_values)

In [46]:
new_df.to_parquet(folder / 'curated/dataset.parquet', engine='pyarrow', compression='snappy', partition_cols=['issue_d'], index=True)

/home/xavi/Documents/CreditRisk/env/lib/python3.12/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
